In [ ]:
import pandas as pd
import numpy as np

Apply the Random Forests algorithm but this time only by upscaling the data.

In [ ]:
categorical = pd.read_csv('files_for_lab/categorical.csv')
numerical = pd.read_csv('files_for_lab/numerical.csv')
target = pd.read_csv('files_for_lab/target.csv')

In [ ]:
#encode the categoricals
from sklearn.preprocessing import OneHotEncoder
encoder = OneHotEncoder(drop='first').fit(categorical)
encoded_categorical = encoder.transform(categorical).toarray()
encoded_categorical = pd.DataFrame(encoded_categorical)

In [ ]:
data = pd.concat([encoded_categorical, numerical, target], axis=1)

In [ ]:
data.head()

In [ ]:
data['TARGET_D'].sum()

In [ ]:
data.TARGET_B.value_counts()

In [ ]:
#upsample
from sklearn.utils import resample
majority_class = data[data.TARGET_B == 0]
minority_class = data[data.TARGET_B == 1]

minority_class_upsampled = resample(minority_class, 
                                   replace=True, 
                                   n_samples = len(majority_class))

data_upsampled = pd.concat([majority_class, minority_class_upsampled], axis=0)

In [ ]:
data_upsampled.shape

In [ ]:
#X-y split
y = data_upsampled['TARGET_B']
X = data_upsampled.drop(['TARGET_B'], axis=1)

#train-test split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

X_train = pd.DataFrame(X_train)
X_test = pd.DataFrame(X_test)

y_train_regression = X_train['TARGET_D']
y_test_regression = X_test['TARGET_D']


X_train = X_train.drop(['TARGET_D'], axis = 1)
X_test = X_test.drop(['TARGET_D'], axis = 1)

In [ ]:
#building the Random Forest classification model
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
clf = RandomForestClassifier(max_depth=5,
                             min_samples_split=20,
                             min_samples_leaf =20)

clf.fit(X_train, y_train)
print('train score: ',clf.score(X_train, y_train))
print('test score: ',clf.score(X_test, y_test))

In [ ]:
#cross validation score
cross_val_scores = cross_val_score(clf, X_train, y_train, cv=10)
print(np.mean(cross_val_scores))

Discuss the output and its impact in the business scenario. Is the cost of a false positive equals to the cost of the false negative? How would you change your algorithm or data in order to maximize the return of the business?

In [ ]:
#confusion matrix
from sklearn.metrics import plot_confusion_matrix

plot_confusion_matrix(clf, X_test, y_test)

The false positives are people that were predicted to donate but did not donate, and the false negatives are the people that were predicted to be non-donors but did actually donate. 

In this case if we only send the promotional mail to the predicted donors the cost of a false negative(missing out on a donation) is far more than the cost of a false positive(the cost of postage). 

It would be best to focus on minimizing the false negatives in order to increase total donation amount. We can lower the threshold for a positive prediction in order to increase the amount of positives predicted. The optimal donor probability threshold for maximum revenue would depend on the cost of postage of each promotional mail(not included in the case study) and the estimated donation amounts(which I will build a model to predict in the next lab).  

In [ ]:
#adding predictions to dataset
y = X['TARGET_D']
X = X.drop(['TARGET_D'], axis=1)

pred = clf.predict(X)
pred_prob = clf.predict_proba(X)

X['predicted_donor'] = pred
X['predicted_donor_prob'] = [pred_prob[i][1] for i in range(len(pred_prob))]
donor_prediction_data = X.copy()
donor_prediction_data.head()

In [ ]:
#find threshold to optimize donations
best_threshold = 0
highest_revenue = 0

for i in range(30,50):
    d = donor_prediction_data[donor_prediction_data['predicted_donor_prob']>=(i/100)]
    cost = .68*len(d)
    donations = d['TARGET_D'].sum()
    revenue = donations - cost
    if revenue > highest_revenue:
        highest_revenue = revenue
        best_threshold = i/100
    
print('highest revenue: ', highest_revenue, ' best threshold: ', best_threshold)

For the current dataset, if we assume that each piece of promotional mail cost 68 cents(I did not see this in the case study documentation but was mentioned in class) and all 95412 potential donors were to receive one, that would cost the company $64880.16. The total amount of donations received was $75668.70, leaving $10788.54 in revenue.

But instead if they had only sent it to the potential donors that the model predicted at least .34 probability of donating, the revenue would have increased to $1284838.32.